In [70]:
import cv2
import numpy as np
import dlib
import os
import glob
import random

In [20]:
from matplotlib import pyplot as plt
%matplotlib inline

In [22]:
predictor5_path = "./shape_predictor_5_face_landmarks.dat"
predictor68_path = "./shape_predictor_68_face_landmarks.dat"
face_rec_model_path = "./dlib_face_recognition_resnet_model_v1.dat"

detector = dlib.get_frontal_face_detector()
sp5 = dlib.shape_predictor(predictor68_path)
sp68 = dlib.shape_predictor(predictor68_path)
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

In [23]:
feats = []

In [71]:
def get_features(im):
    dets = detector(im, 1)
    if len(dets) == 0:
        return None  # no face detected :(
    for d in dets:
        # TODO: what if there are two faces in one pic --> seems ok
#         print(d)
        cur_feat = []
        x_l = d.left()
        x_r = d.right()
        y_t = d.top()
        y_b = d.bottom()


        width = x_r - x_l
        height = y_b - y_t
    #     cv2.rectangle(orig, (x, y), (x + w, y + h), (0, 255, 0), im.shape[0]//150)
        shape = sp68(im, d)
        for i in range(0, 68):
            # TODO: normalize to 0 mean? How?
            feat_x = (shape.part(i).x - x_l) / width
            feat_y = (shape.part(i).y - y_t) / height
            cur_feat.append(feat_x)
            cur_feat.append(feat_y)

    #         cv2.circle(orig, (shape.part(i).x, shape.part(i).y), 1, (255, 0, 0), im.shape[0]//150)
    #     feats.append(cur_feat)
    return cur_feat

In [35]:
# plt.imshow(cv2.cvtColor(orig, cv2.COLOR_BGR2RGB))

In [25]:
from sklearn import svm

In [26]:
X_train  = feats
y = [0, 1]

In [27]:
clf = svm.LinearSVC() # TODO: consider non-linear kernels: svm.SVC()

In [28]:
clf.fit(X_train, y)

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
clf.predict([feats[1], feats[0]])

In [30]:
emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"] #Emotion list

In [36]:
def get_files(emotion): #Define function to get file list, randomly shuffle it and split 80/20
    files = glob.glob("google_dataset\\%s\\*" %emotion)
    random.shuffle(files)
    training = files[:int(len(files)*0.8)] #get first 80% of file list
    prediction = files[-int(len(files)*0.2):] #get last 20% of file list
    return training, prediction

In [41]:
def make_sets():
    training_data = []
    training_labels = []
    prediction_data = []
    prediction_labels = []
    for emotion in emotions:
        training, prediction = get_files(emotion)
        #Append data to training and prediction list, and generate labels 0-7
        for item in training:
            image = cv2.imread(item) #open image
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #convert to grayscale
            current_feats = get_features(gray)
            if current_feats:
                training_data.append(current_feats) #append image feats to training data list
                training_labels.append(emotions.index(emotion))
    
        for item in prediction: #repeat above process for prediction set
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            current_feats = get_features(gray)
            if current_feats:
                prediction_data.append(current_feats)
                prediction_labels.append(emotions.index(emotion))

    return training_data, training_labels, prediction_data, prediction_labels

In [42]:
sets_made = make_sets()

[(52, 52) (319, 320)]
[(52, 52) (319, 320)]
[(27, 63) (348, 384)]
[(22, 52) (290, 320)]
[(27, 63) (348, 384)]
[(27, 63) (348, 384)]
[(22, 52) (290, 320)]
[(52, 52) (319, 320)]
[(52, 82) (319, 350)]
[(22, 52) (290, 320)]
[(27, 63) (348, 384)]
[(52, 52) (319, 320)]
[(22, 52) (290, 320)]
[(52, 52) (319, 320)]
[(27, 63) (348, 384)]
[(52, 52) (319, 320)]
[(27, 63) (348, 384)]
[(27, 27) (348, 348)]
[(27, 63) (348, 384)]
[(52, 82) (319, 350)]
[(52, 82) (319, 350)]
[(52, 52) (319, 320)]
[(27, 63) (348, 384)]
[(27, 27) (348, 348)]
[(27, 63) (348, 384)]
[(22, 52) (290, 320)]
[(52, 82) (319, 350)]
[(52, 52) (319, 320)]
[(52, 52) (319, 320)]
[(52, 52) (319, 320)]
[(52, 52) (319, 320)]
[(22, 52) (290, 320)]
[(52, 82) (319, 350)]
[(22, 52) (290, 320)]
[(52, 52) (319, 320)]
[(52, 82) (319, 350)]
[(52, 52) (319, 320)]
[(27, 63) (348, 384)]
[(52, 52) (319, 320)]
[(52, 82) (319, 350)]
[(52, 52) (319, 320)]
[(27, 63) (348, 384)]
[(27, 63) (348, 384)]
[(27, 63) (348, 384)]
[(52, 82) (319, 350)]
[(52, 52) 

In [139]:
def run_recognizer():
    training_data, training_labels, prediction_data, prediction_labels = make_sets()
#     training_data, training_labels, prediction_data, prediction_labels = sets_made  # to speed up TEMP
    
    print ("training SVM classifier")
    print ("size of training set is:", len(training_labels), "images")
    print ("size of test set is:", len(prediction_labels), "images")
    
    X = training_data
    y = training_labels
    
#     clf = svm.LinearSVC(C=5)
    clf = svm.SVC(C=0.8, kernel='rbf', gamma=20)  # it's worse
    clf.fit(X, y)
    print ("predicting classification set")
    cnt = 0
    correct = 0
    incorrect = 0
    
    for image in training_data:
        pred = clf.predict([image]) #predict emotion
        if pred == training_labels[cnt]: #validate it
            correct += 1
            cnt += 1
        else:
            incorrect += 1
            cnt += 1
    print("Training accuracy:", ((100*correct)/(correct + incorrect))) 

    cnt = 0
    correct = 0
    incorrect = 0
    for image in prediction_data:
        pred = clf.predict([image]) #predict emotion
        if pred == prediction_labels[cnt]: #validate it
            correct += 1
            cnt += 1
        else:
            incorrect += 1
            cnt += 1
    return ((100*correct)/(correct + incorrect))

In [140]:
# run_recognizer()

In [141]:
#Now run it
metascore = []
for i in range(0,3):
    correct = run_recognizer()
    print ("Test: got", correct, "percent correct!")
    metascore.append(correct)

print ("\n\nend score:", np.mean(metascore), "percent correct!")

training SVM classifier
size of training set is: 132 images
size of test set is: 30 images
predicting classification set
Training accuracy: 94.6969696969697
Test: got 26.666666666666668 percent correct!
training SVM classifier
size of training set is: 130 images
size of test set is: 31 images
predicting classification set
Training accuracy: 93.84615384615384
Test: got 25.806451612903224 percent correct!
training SVM classifier
size of training set is: 131 images
size of test set is: 31 images
predicting classification set
Training accuracy: 93.89312977099236
Test: got 29.032258064516128 percent correct!


end score: 27.1684587814 percent correct!
